In [ ]:
import pandas as pd

#df = pd.read_csv("data/compacted_rules.csv")
df = pd.read_csv("mlpipe/exstracs_train_full/compacted_rules.csv")
df_report = pd.read_csv("data/text_reports.csv")

In [ ]:
df

In [ ]:

df_report


In [ ]:
df_report = df_report.replace({0.25:1, 0.5:1, 0.75:1})
df_report = df_report.replace({"COM_CCD":"COM", "COM_MMM":"COM", "NM_CAP":"NM", "CFTD":"OTHER", "NON_CM":"OTHER","CM":"UNCLEAR"})
count_report_dict_attrb = df_report.count().to_dict()
count_report_dict_concl = df_report["conclusion"].value_counts().to_dict()

### Definition des paramètres du réseau d'attributs
* Noeuds: Attributs ; couleur par sommes de ses numerosité * accuracy
* Tailles des Noeuds: Nb de fois qu'on le retrouve
* Lien: Si dans la même règle + thickness par occurence, couleur par numérosité * accuracy

### Definition des paramètres du réseau attributs - conclusion
* Noeuds: Attributs + 1 par conclusions, couleur noeud: par coloration
* Tailles des Noeuds: Nb de fois qu'on le retrouve (cb de règles)
* Lien: Lien conclusions - attributs avec thickness par occurence, couleur par numérosité * accuracy

### Algo:
Liste d'attributs unique, créer les noeuds
Puis créer les liens en ittérant par ligne du tableau de règles
Si le lien existe déjà: modifier la taille du noeuds ?

Tableau: id, valeur, titre, label, couleur

In [ ]:
import json
onto = json.load(open("data/ontology.json", "r"))
def id_to_name(onto, ids_list, mode="full"):
    """Map a list of ID of ontology term to their names
    Args:
        onto (list): ontology json as list
        ids_list (_type_): list of ID from ontology to map
        mode (str, optional): Mapping mode (id+name or only name) Defaults to "full".
    Returns:
        name_list list : list of nodes names
    """
    json_index = {}
    name_list = []
    for i in onto:
        json_index[i["id"]] = i
    if mode == "full":
        if isinstance(ids_list, list):
            for id in ids_list:
                try:
                    name_list.append(id + " " + json_index[id]["text"])
                except:
                    name_list.append(id)
        elif isinstance(ids_list, str):
            try:
                name_list = ids_list + " " + json_index[ids_list]["text"]
            except:
                name_list = ids_list
    if mode == "short":
        if isinstance(ids_list, list):
            for id in ids_list:
                try:
                    name_list.append(json_index[id]["text"])
                except:
                    name_list.append(id)
        elif isinstance(ids_list, str):
            try:
                name_list = json_index[ids_list]["text"]
            except:
                name_list = ids_list
    return name_list

In [ ]:
# TODO: Taille des noeuds et des maladies par la quantité dans notre jeu de données de base

# All Attrib Nodes
from collections import Counter
import itertools

full_rules_attrb_split = [] 
for i in df["Specified Attribute Names"].to_list():
    for j in i.split(", "):
        full_rules_attrb_split.append(j)
unique_attrb_count = Counter(full_rules_attrb_split)

# All Attrib Edges
all_edges = []
for i in df["Specified Attribute Names"].to_list():
    rule_attrib_list = i.split(", ")
    if len(rule_attrib_list) > 1:
        sorted_attrib = sorted(rule_attrib_list)
        edges = list(itertools.combinations(rule_attrib_list, 2))
        for edge in edges:
            all_edges.append(edge)
all_edges_count = Counter(all_edges)

# All Conclusions Nodes
full_rules_concl_split = [] 
name_dict = {0: "Centronuclear Myopathy", 1: "Core Myopathy", 2: "Nemaline Myopathy"}
for i in df["conclusion"].to_list():
    full_rules_concl_split.append(name_dict[i])
unique_concl_count = Counter(full_rules_concl_split)

# All Conclusions Edges IF value is different from 0.
all_edges_concl = []
all_edges_color = {}

for index, row in df.iterrows():
    rule_attrib_list = row[1].split(", ")
    rule_attrib_value = row[0].split(", ")
    sorted_attrib = sorted(rule_attrib_list)
    for index, value in enumerate(sorted_attrib):
        all_edges_concl.append((name_dict[row[2]], value))
        # Set list of edge color depending of attrib status
        previous_color = all_edges_color.get((name_dict[row[2]], value), None)
        # Probably a dumb way of doing it... yet.
        try:
            current_value = int(float(rule_attrib_value[index]))
        except:
            current_value = None
        if current_value == None:
            all_edges_color[(name_dict[row[2]], value)] = "#f4b400"
        elif previous_color is None:
            if current_value <= 0:
                all_edges_color[(name_dict[row[2]], value)] = "#db4437"
            elif current_value > 0:
                all_edges_color[(name_dict[row[2]], value)] = "#0f9d58"
        else:
            if current_value <= 0 and previous_color != "#db4437":
                all_edges_color[(name_dict[row[2]], value)] = "#f4b400"
            elif current_value > 0 and previous_color != "#0f9d58":
                all_edges_color[(name_dict[row[2]], value)] = "#f4b400"

all_edges_conl_count = Counter(all_edges_concl)

# All Conclusions Edges IF value is different from 0.
all_edges_concl_nnull = []

for index, row in df.iterrows():
    rule_attrib_list = row[1].split(", ")
    rule_attrib_value = row[0].split(", ")
    sorted_attrib = sorted(rule_attrib_list)
    try:
        current_value = int(float(rule_attrib_value[index]))
    except:
        current_value = None
    for index, value in enumerate(sorted_attrib):
        if current_value != 0:
            all_edges_concl_nnull.append((name_dict[row[2]], value))
all_edges_conl_count_nnull = Counter(all_edges_concl_nnull)

# All Attrib NodesIF value is different from 0.
full_rules_attrb_nnull_split = [] 
for index, row in df.iterrows():
    rule_attrib_list = row[1].split(", ")
    rule_attrib_value = row[0].split(", ")
    sorted_attrib = sorted(rule_attrib_list)
    try:
        current_value = int(float(rule_attrib_value[index]))
    except:
        current_value = None
    for index, value in enumerate(sorted_attrib):
        if current_value != 0:
            full_rules_attrb_nnull_split.append(value)
    unique_attrb_count_nnull = Counter(full_rules_attrb_nnull_split)

In [ ]:
from pyvis import network as net
import networkx as nx

#net = Network(height='100%', width='100%', bgcolor='#222222', font_color='white', notebook=True)
net_1 = net.Network(height='95%', width='70%', notebook=True)
for key, value in unique_attrb_count.items():
    net_1.add_node(key, value=value)  
for key, value in all_edges_count.items():
    net_1.add_edge(key[0], key[1], width=value*3, length=300)
#net.show_buttons(filter_=["physics"])
net_1.set_options("""
var options = {
  "layout": {
    "randomSeed": 777777
    },
  "edges": {
      "smooth": true
    }
}
""")
net_1.show("example.html")


In [ ]:
from pyvis import network as net
import networkx as nx

#net = Network(height='100%', width='100%', bgcolor='#222222', font_color='white', notebook=True)
net_2 = net.Network(height='95%', width='70%', notebook=True)
for key, value in unique_attrb_count_nnull.items():
    net_2.add_node(key, value=value, color="#4885ed")
for key, value in unique_concl_count.items():
    net_2.add_node(key, value=value, color="#db3236", shape="triangle")  
for key, value in all_edges_conl_count_nnull.items():
    net_2.add_edge(key[0], key[1], width=value, length=200, color="#f4c20d")
#net_2.show_buttons(filter_=["physics"])
net_2.set_options("""
var options = {
  "layout": {
    "randomSeed": 777777
    },
  "edges": {
      "smooth": true
    }
}
""")
net_2.show("example2.html")

In [ ]:
from pyvis import network as net
import networkx as nx

#net = Network(height='100%', width='100%', bgcolor='#222222', font_color='white', notebook=True)
net_3 = net.Network(height='95%', width='70%', notebook=True)
for key, value in unique_attrb_count.items():
    #net_3.add_node(key, value=value, color="#4885ed", size=count_report_dict_attrb[key])
    net_3.add_node(key, label=id_to_name(onto, key, "short"), value=value, color="#4885ed")
for key, value in unique_concl_count.items():
    net_3.add_node(key, label=id_to_name(onto, key, "short"), value=value, color="#BA55D3", shape="triangle")  
for key, value in all_edges_conl_count.items():
    net_3.add_edge(key[0], key[1], width=value, length=250, color=all_edges_color[(key[0],key[1])])
#net_3.show_buttons(filter_=["physics"])
net_3.set_options("""
var options = {
  "layout": {
    "randomSeed": 777777
    },
  "edges": {
      "smooth": true
    }
}
""")
net_3.show("example3.html")

In [ ]:
all_edges_color